Build a constellation of Planet Dove satellites

In [65]:
import time
import numpy as np
import requests
from sedaro import SedaroApiClient
from sedaro_base_client.apis.tags import jobs_api

# Settings
API_KEY = '' 
SCENARIO_BRANCH_ID = 'NXVzrUNTFH9pNn8zTSH1F'
HOST = 'https://api.sedaro.com'

BASE_AGENT_ID = 'NT-KoZFSELKK8eomP3lkV'

delete satelllite with old TLEs

In [66]:
with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
    scenario = sedaro_client.get_branch(SCENARIO_BRANCH_ID)

    delete_ids=[]
    for agent_id in scenario.data['index']['Agent']:
        if agent_id != BASE_AGENT_ID and not (scenario.data['blocks'][agent_id]['peripheral']):
            
            delete_ids.append(agent_id)

    if not delete_ids: 
        print('no blocks were deleted')
        
    else:
        scenario.crud(delete=delete_ids)
        print(f'deleted {len(delete_ids)} spacecraft digital twin agents.')

    

no blocks were deleted


create new agents with new TLEs

In [67]:
with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
    scenario = sedaro_client.get_branch(SCENARIO_BRANCH_ID)

    base = scenario.Agent.get(BASE_AGENT_ID)

    def create_orbit(tle):
        _orbit = {
            **base.orbit.data,
            'initialStateDefParams': {'tle': tle}
        }
        _orbit.pop('id')

        return _orbit

    def create_agent(name, orbit_id):
        temp = {**base.data, 'orbit': orbit_id, 'name': name}
        del temp['id']
        
        return temp

    # Load TLE file
    url = 'https://ephemerides.planet-labs.com/planet_mc.tle'
    tles = requests.get(url).text.splitlines()

    orbits=[]
    n_sats = int(len(tles)/3)   #temporary limit of to the ammmount of sats in constellation due to issue with simulating all 218
    for idx in range(0, n_sats):
        orbits.append(create_orbit( tles[3*idx+1] + '\n' + tles[3*idx+2]))

    orbit_ids = scenario.crud(blocks=orbits)['crud']['blocks']

    print(f'Created {n_sats} orbits from tles.')

    agents=[]
    for idx in range(0, n_sats):
        agents.append(create_agent(tles[3*idx], orbit_ids[idx]))

    scenario.crud(blocks=agents)

    print(f'Created {n_sats} spacecraft digital twin agents.')

Created 218 orbits from tles.
Created 218 spacecraft digital twin agents.
